<a href="https://colab.research.google.com/github/sheilsarda/ESE546_Final_Project/blob/master/CAML_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""MAML+Mujoco

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1pUx0onzAdHv2lHEl9BUQ8SF-MH992_Ol

## **Simulation Environment is from CS330_fall2020**

The following simulation environment is from a homework stencil from Stanford's CS330_fall2020: Goal Conditioned Reinforcement Learning and Hindsight Experience Replay.

https://colab.research.google.com/drive/1uYqzq4Ix91DD4QdElyrbC_If0TKDf_3t?usp=sharing#scrollTo=5yTRSgI-ryd-
"""

# Commented out IPython magic to ensure Python compatibility.
import os
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@title set up mount symlink

DRIVE_PATH = '/content/gdrive/My\ Drive/finalproj'
DRIVE_PYTHON_PATH = DRIVE_PATH.replace('\\', '')
if not os.path.exists(DRIVE_PYTHON_PATH):
  %mkdir $DRIVE_PATH

## the space in `My Drive` causes some issues,
## make a symlink to avoid this
SYM_PATH = '/content/finalproj'
if not os.path.exists(SYM_PATH):
  !ln -s $DRIVE_PATH $SYM_PATH

#@title Install Requirements
#@markdown Requirements for the assignment and display drivers

# Robot sim
!pip install gym==0.15.4
!pip install pygame

# Various things for render
!apt-get install python-opengl -y
!apt install xvfb -y

# Rendering Environment
!pip install pyvirtualdisplay
!pip install piglet
!sudo apt-get install -y xvfb ffmpeg
!pip install imageio
!pip install PILLOW

# Commented out IPython magic to ensure Python compatibility.
#@title Download Mujoco from an online repository

MJC_PATH = '{}/mujoco'.format(SYM_PATH)
if not os.path.exists(MJC_PATH):
  %mkdir $MJC_PATH
%cd $MJC_PATH
if not os.path.exists(os.path.join(MJC_PATH, 'mujoco200')):
  !wget -q https://www.roboti.us/download/mujoco200_linux.zip
  !unzip -q mujoco200_linux.zip
#   %mv mujoco200_linux mujoco200
#   %rm mujoco200_linux.zip

#@title Important: ACTION Required BEFORE running this cell
#@markdown Place the mujoco key we have given you into a text file called mjkey.txt 
#@markdown and ensure that the mujoco key is in the Google Drive path `cs330_fall2020/mujoco`.

import os

os.environ['LD_LIBRARY_PATH'] += ':{}/mujoco200/bin'.format(MJC_PATH)
os.environ['MUJOCO_PY_MUJOCO_PATH'] = '{}/mujoco200'.format(MJC_PATH)
os.environ['MUJOCO_PY_MJKEY_PATH'] = '{}/mjkey.txt'.format(MJC_PATH)

## installation on colab does not find *.so files
## in LD_LIBRARY_PATH, copy over manually instead
!cp $MJC_PATH/mujoco200/bin/*.so /usr/lib/x86_64-linux-gnu/

#@title Important system updates for mujoco-py
!apt update 
!apt install -y --no-install-recommends \
        build-essential \
        curl \
        git \
        gnupg2 \
        make \
        cmake \
        ffmpeg \
        swig \
        libz-dev \
        unzip \
        zlib1g-dev \
        libglfw3 \
        libglfw3-dev \
        libxrandr2 \
        libxinerama-dev \
        libxi6 \
        libxcursor-dev \
        libgl1-mesa-dev \
        libgl1-mesa-glx \
        libglew-dev \
        libosmesa6-dev \
        lsb-release \
        ack-grep \
        patchelf \
        wget \
        xpra \
        xserver-xorg-dev \
        xvfb \
        python-opengl \
        ffmpeg > /dev/null 2>&1

# Commented out IPython magic to ensure Python compatibility.
#@title Clone and install mujoco-py
#@markdown Remember that you need to put the key in the appropriate location as described above
# %cd $MJC_PATH
if not os.path.exists('mujoco-py'):
  !git clone https://github.com/openai/mujoco-py.git
# %cd mujoco-py
# %pip install -e .

## cythonize at the first import
import mujoco_py


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
xvfb is already the newest version (2:1.19.6-1ubuntu4.8).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
/content/gdrive/My Drive/finalproj/mujoco
cp: cannot stat '/content/finalproj/mujoco/mujoco200/bin/*.so': No such file or directory
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cra

In [25]:
....."""
installing dependencies
"""
!apt-get -qq -y install libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

In [27]:
"""
Imports
"""
import gym
import torch
import numpy as np
import pickle
import argparse 
import time 
import gym
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
import random
from gym import wrappers

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
# os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

import matplotlib.animation
import numpy as np
from IPython.display import HTML

# Note, you may get a few warnings regarding Tensorflow and xdpyinfo, these are to be expected

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import random
from torch.nn.modules.loss import CrossEntropyLoss
from random import shuffle
import sys
from copy import deepcopy
import warnings
import math, random
import gym
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd  
import torch.nn.functional as F
import abc
import collections
import numpy as np
import torch
from torch import nn
from torch import distributions as td
from torch.nn import functional as F
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim
from torch.distributions import Categorical
import gym
from collections import deque
import numpy as np 
from torch.distributions import Categorical
import matplotlib.pyplot as plt

In [30]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


In [29]:
class Policy(nn.Module): 
    def __init__(self, s_size=6, h_size=50, a_size=3):
        super(Policy, self).__init__()
        self.l1 = nn.Linear(s_size, h_size)
        self.l2 = nn.Linear(h_size, a_size)

        self.model = nn.Sequential(
            self.l1, 
            nn.ReLU(), 
            self.l2, 
            nn.Softmax(dim=1)
        )
         
    def forward(self, x):
        return self.model(x)
    
    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item() - 1, m.log_prob(action)

def compute_rewards(rewards, gamma):
    discounted_rewards = np.zeros(len(rewards))
    moving_add = 0
    for i in reversed(range(0, len(rewards))):
        moving_add = moving_add*gamma + rewards[i]
        discounted_rewards[i] = moving_add

    return discounted_rewards

class Critic(nn.Module): 
    def __init__(self, state_dim=6, hidden_dim=20, output_dim=1, lambd=.9):
        super(Critic, self).__init__()
        self.l1 = nn.Linear(state_dim, hidden_dim, bias=False)
        self.l2 = nn.Linear(hidden_dim, output_dim, bias=False)
        self.lambd = lambd

    def forward(self, state): 
        state = torch.from_numpy(state).float().unsqueeze(0) 
        x = F.relu(self.l1(state)) 
        x = self.l2(x) 
        return F.softmax(x, dim=1)

    def td_error(self, reward, value_next, value_now, gamma, done, I): 
        if done: I = I * gamma 
        td_error = reward + gamma*(1-done)*value_next - value_now 
        return td_error 
        
def train(n_episodes, policy, critic, gamma, print_every=4):
    optimizer = optim.Adam(policy.parameters(), lr=0.001)
    optimizer_v = optim.Adam(critic.parameters(), lr=.001)
    scores_deque = deque(maxlen=100)
    
    total_rewards = []
    for ep in range(n_episodes): 
        traj_log_probs = []
        rewards = []
        state = env.reset()
        score = 0 
        I = 1.0 
        done = False
 
        while not done:
            action, log_prob = policy.act(state)
            #value_func = critic.forward(state)
            traj_log_probs.append(log_prob)
            next_state, reward, done, _ = env.step(action)
            #value_func_next = critic(next_state)
            #td_error = critic.td_error(reward, value_func_next, value_func, gamma, done, I) 
            
            score += reward
            rewards.append(reward)
           
        scores_deque.append(score) 
        total_rewards.append(score)
                
        disc_rewards = compute_rewards(rewards, gamma)
        disc_rewards = torch.tensor(disc_rewards)
        
        policy_loss = [] 
        for t, log_prob in enumerate(traj_log_probs):
            policy_loss.append(-log_prob * disc_rewards[t])  
        policy_loss = torch.cat(policy_loss).sum() 
        
        #value_loss = F.l1_loss(value, torch.tensor([disc_rewards]))
        #add gradient trace 
        #for p in critic.parameters(): 
        #    p.grad = p.grad * critic.lambd
        
        #loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()

        # backprop
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()
        
        if ep % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(ep, np.mean(scores_deque)))        
    
    return total_rewards

if __name__ == "__main__":
    start_time = time.time()
    envName = 'HalfCheetah-v2'
    print("Cuda available: {}".format(torch.cuda.is_available()))
    if torch.cuda.is_available():
        print("Device Count: {}".format(torch.cuda.device(0)))
        print("Device Name (first): {}".format(torch.cuda.get_device_name(0)))
    print("Environment Used: {}".format(envName))
    env = gym.make(envName) 
    action_space = env.action_space.shape[0]
    state_space = env.observation_space.shape[0] 
    hp = {'tau': 0.005,\
            'lr': 3*(1e-4),\
            'batch_size': 256,\
            'gamma': 0.99,\
            'max_action': env.action_space.high,\
            'min_action': env.action_space.low\
            }


Cuda available: False
Environment Used: HalfCheetah-v2


DependencyNotInstalled: ignored

In [4]:
class DQN_Network(nn.Module):
    def __init__(self): 
        super(DQN_Network, self).__init__() 
        
        # __GRU__ 
        """
        self.input_layer = nn.Linear(8, 128)
        self.hidden_1 = nn.Linear(128, 128)
        self.hidden_2 = nn.Linear(32,31)
        self.hidden_state = torch.tensor(torch.zeros(2,1,32))
        self.rnn = nn.GRU(128, 32, 2)
        self.action_head = nn.Linear(31, 5)
        """
        
        self.input_layer = nn.Linear(8, 32)
        self.hidden_1 = nn.Linear(32, 32)
        self.hidden_2 = nn.Linear(32,31)
        self.output_layer = nn.Linear(31, 5)
    

    def forward(self, state):
        state = state.squeeze()

        """
        out = torch.sigmoid(self.input_layer(state))
        out = torch.tanh(self.hidden_1(out))
        out, self.hidden_state = self.rnn(out.view(1,-1,128), self.hidden_state.data)
        out = F.relu(self.hidden_2(out.squeeze()))
        out = self.action_head(out)
        """
        out = F.relu(self.input_layer(state))
        out = F.relu(self.hidden_1(out))
        out = F.relu(self.hidden_2(out))
        out = F.sigmoid(self.output_layer(out))

        """
        out = F.relu(self.input_layer(state))
        out = F.relu(self.hidden_1(out))
        out = F.relu(self.hidden_2(out))
        out = F.relu(self.output_layer(out))
        """
        return out 

class DQN_Agent(): 
    def __init__(self): 
        #initialize target and policy networks 
        self.target_net = DQN_Network()
        self.policy_net = DQN_Network()

        self.eps_start = .9  
        self.eps_end = .05
        self.eps_decay = 200  
        self.steps_done = 0 

    def select_action(self, state): 
        random_n = random.random() #generate random number
        eps_threshold = self.eps_end + (self.eps_start - self.eps_end) * \
            math.exp(-1. * self.steps_done / self.eps_decay)
        self.steps_done += 1  

        if (random_n < eps_threshold): 
        #take random action (random # betwee 0 and 4)
            action = torch.tensor([random.randrange(4)]) 
        else: 
        #take the best action  
            with torch.no_grad(): 
                actions = self.policy_net(state)  
                action = torch.argmax(actions).view(1, 1) 
                
        return action.item() 
